In [2]:
%pip install sklearn
%pip install sqlite3

Defaulting to user installation because normal site-packages is not writeable
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
Note: you may need to restart the kernel to use updated packages.


    ERROR: Command errored out with exit status 1:
     command: 'c:\Program Files\Python39\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\user\\AppData\\Local\\Temp\\pip-install-aysmi94n\\sklearn\\setup.py'"'"'; __file__='"'"'C:\\Users\\user\\AppData\\Local\\Temp\\pip-install-aysmi94n\\sklearn\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\user\AppData\Local\Temp\pip-pip-egg-info-xz6dbc8k'
         cwd: C:\Users\user\AppData\Local\Temp\pip-install-aysmi94n\sklearn\
    Complete output (15 lines):
    The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
    rather than 'sklearn' for pip commands.
    
    Here is how to fix this error in the main use cases:
    - use 'pip install scikit-learn' rather than 'pip install sklearn'
    - replace 'sklearn' by 'scikit-learn' in your pip requireme

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3
You should consider upgrading via the 'c:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


In [5]:
%pip install catboost

Defaulting to user installation because normal site-packages is not writeable
  Using cached plotly-5.24.1-py3-none-any.whl (19.1 MB)
  Using cached graphviz-0.20.3-py3-none-any.whl (47 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl (28 kB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


In [9]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
import numpy as np

def predict_parking_purchase_probability_catboost(filepath, client_id_col='client_id', target_col='target', test_size=0.2, random_state=42):
    try:
        df = pd.read_csv(filepath)

        if client_id_col not in df.columns or target_col not in df.columns:
            print(f"Error: Columns '{client_id_col}' and '{target_col}' not found in '{filepath}'.")
            return None, None, None

        # Separate features and target
        X = df.drop(columns=[client_id_col, target_col])
        y = df[target_col]

        # Identify categorical features and handle potential numerical values in categorical columns.
        categorical_features = X.select_dtypes(include=['object', 'category']).columns
        for col in categorical_features:
            #Convert to string if not already string, handle NaNs, and ensure no numeric values
            if X[col].dtype != 'object':
                X[col] = X[col].astype(str)
            X[col] = X[col].fillna('Unknown')
            #If any values are numbers after above changes, force them to strings.
            X[col] = X[col].apply(lambda x: str(x) if isinstance(x, (int, float)) else x )


        # Get indices of categorical features for CatBoost
        categorical_features_indices = np.where(X.dtypes == 'object')[0]

        # Split data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

        # Initialize and train the CatBoost model
        model = CatBoostClassifier(iterations=4819,
                                   learning_rate=0.1,
                                   loss_function='Logloss',
                                   random_seed=random_state,
                                   verbose=100,
                                   early_stopping_rounds=4819)

        model.fit(X_train, y_train, cat_features=categorical_features_indices, eval_set=(X_test, y_test))

        # Make predictions (probabilities)
        probabilities = model.predict_proba(X_test)[:, 1]

        # Evaluate the model
        accuracy = accuracy_score(y_test, model.predict(X_test))
        roc_auc = roc_auc_score(y_test, probabilities)
        report = classification_report(y_test, model.predict(X_test))

        evaluation_metrics = {
            "accuracy": accuracy,
            "roc_auc": roc_auc,
            "classification_report": report
        }

        return model, probabilities, evaluation_metrics

    except FileNotFoundError:
        print(f"Error: File '{filepath}' not found.")
        return None, None, None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None, None, None

# Example Usage (Remember to replace 'your_file.csv' with your actual file path)
filepath = 'train_data_clients.csv'
model, probabilities, evaluation_metrics = predict_parking_purchase_probability_catboost(filepath)

if model:
    print("Model trained successfully.")
    print("Evaluation Metrics:", evaluation_metrics)
    print("Probabilities:", probabilities)


C:\Users\user\AppData\Local\Temp\ipykernel_14960\2437872949.py:9: DtypeWarning: Columns (51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,87,88,89,90,91,92,93,95,96,97,99,100,101,131,132,135,136,139,140,141,143,144,147,148,151,152,155,156,157,159,160,163,164,167,168,171,172,173,174,175,176,177,183,184,185,187,188,189,191,192,193,194,195,196,197,198,203,204,205,207,208,209,211,212,213,214,215,216,217,223,224,239,240,241,243,244,459,467,515,523,531,539,547,555,563,571,579,580,581,583,587,595,603,611,612,615,795,799,1031,1032,1033,1034,1035,1036,1039,1040,1041,1043,1044,1047,1048,1049,1051,1052,1055,1056,1057,1059,1060,1063,1064,1065,1067,1068,1072,1073,1074,1075,1076,1077,1174,1176,1177,1178,1179,1180,1181,1182,1275,1276,1277,1278,1279,1280,1281,1282,1447,1448,1449,1450,1451,1452,1453,1454,1651,1652,1653,1654,1656,2193,2194,2195,2196,2199) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


0:	learn: 0.5440060	test: 0.5457394	best: 0.5457394 (0)	total: 683ms	remaining: 54m 48s
100:	learn: 0.0791582	test: 0.1197345	best: 0.1195092 (89)	total: 4m 14s	remaining: 3h 17m 47s
200:	learn: 0.0551363	test: 0.1214472	best: 0.1188417 (106)	total: 8m 40s	remaining: 3h 19m 29s
300:	learn: 0.0389388	test: 0.1267611	best: 0.1188417 (106)	total: 12m 51s	remaining: 3h 12m 54s
400:	learn: 0.0289858	test: 0.1310640	best: 0.1188417 (106)	total: 17m 25s	remaining: 3h 12m 1s
500:	learn: 0.0223561	test: 0.1368879	best: 0.1188417 (106)	total: 21m 38s	remaining: 3h 6m 28s
600:	learn: 0.0175532	test: 0.1393058	best: 0.1188417 (106)	total: 26m	remaining: 3h 2m 28s
700:	learn: 0.0140116	test: 0.1426796	best: 0.1188417 (106)	total: 30m 12s	remaining: 2h 57m 26s
800:	learn: 0.0114246	test: 0.1466415	best: 0.1188417 (106)	total: 34m 22s	remaining: 2h 52m 25s
900:	learn: 0.0099033	test: 0.1498565	best: 0.1188417 (106)	total: 38m 30s	remaining: 2h 47m 25s
1000:	learn: 0.0083830	test: 0.1532507	best: 0.11